In [1]:
import pandas as pd

In [10]:
def cleanup(k):
    dirty_path = 'yelp_dataset/yelp_academic_dataset_review.json'
    clean_path = "cleaned/review%i.json" % k
    dirty_file = open(dirty_path, "r")
    clean_file = open(clean_path, "w")
    
    start = 1000000 * k
    end = 1000000 * (k+1)
    
    content = ''
    i = 0
    for line in dirty_file:
        if i == end:
            break
        elif i >= start:
            s = line.replace('\n', ',\n')
            content += s
        i += 1
    payload = '{"data" : \n[%s]}' % (content[:-2] + '\n')
    clean_file.write(payload)

In [9]:
for k in range(6):
    cleanup(k)
    print('Iteration', k, 'done')

break
Iteration 0 done
break
Iteration 1 done
break
Iteration 2 done
break
Iteration 3 done
break
Iteration 4 done
Iteration 5 done


## Get all restaurants in Toronto (CA)

In [13]:
biz = pd.read_csv('cleaned_csv/restaurants.csv').drop(['is_open', 'hours', 'Unnamed: 0'], axis=1)

In [15]:
biz.city = biz.city.str.lower()
biz.head()

,business_id,city,latitude,longitude,name,postal_code,review_count,stars,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,calgary,51.091813,-114.031675,Minhas Micro Brewery,T2E 6L6,24,4.0,AB
1,AjEbIBw6ZFfln7ePHha9PA,henderson,35.960734,-114.939821,CK'S BBQ & Catering,89002,3,4.5,NV
2,O8S5hYJ1SMc8fA4QBtVujA,montréal,45.540503,-73.599300,La Bastringue,H2G 1K7,5,4.0,QC
3,45bWSZtniwPRiqlivpS8Og,phoenix,33.671375,-112.030017,The Coffee Bean & Tea Leaf,85024,63,4.0,AZ
4,9A2quhZLyWk0akUetBd8hQ,toronto,43.664378,-79.414424,Bnc Cake House,M6G 1K8,7,4.0,ON


In [22]:
biz_toronto = biz[biz.city == 'toronto']
print("Rstaurants in Toronto: ", biz_toronto.shape[0])

Rstaurants in Toronto:  9678


In [18]:
biz_toronto.to_csv('toronto/restaurants_toronto.csv')

## Get all reviews from Toronto

In [2]:
n = 6
df_list = [None] * n
for i in range(n):
    path = "cleaned/review%i.json" % i
    df_list[i] = pd.DataFrame(list(pd.read_json(path).data))    

In [3]:
DF = pd.concat(df_list)

In [5]:
len(DF)

5996996

In [19]:
toronto_reviews = DF[DF.business_id.isin(biz_toronto.business_id)]

In [23]:
print("Reviews from Toronto: ", toronto_reviews.shape[0])

Reviews from Toronto:  379875


In [9]:
toronto_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
134,sscP36MO34i5ep4QO8c7KA,1,2012-01-28,0,dp6rinVjRXYaAABTU6BYsA,3,Sometimes I ride the 501 streetcar just for so...,1,YHWsLBS8jzZiPjKHMFOaAA
139,BmgfYQ3tpRKnT9H9WvjL_g,2,2011-02-23,1,BqMTHIJ_S5aT3Mt1fWT_pg,3,"As I'm writing this, I already know a lot of p...",7,YHWsLBS8jzZiPjKHMFOaAA
156,HX0QhTVWztcawNof0KUhgQ,3,2011-04-17,1,kUgUo3SfZDj_XwAECQtkjQ,3,"Despite it's seemingly religious name, this Bl...",5,YHWsLBS8jzZiPjKHMFOaAA
324,WYG7xp_UO12qf5_qRLBQHw,1,2010-10-24,1,xjUv6EeU_6wRbfmoW_oxoA,3,Since this place is one of the Indie Coffee Pa...,1,YgavGxfAdjhkkbwlAY_9ZQ
327,nhfZW6Pi-oLQuhYcrcZJWw,4,2010-10-05,0,sQlDlTNl8qSciIUQOzFdUw,5,I ordered a soy latte and it was PERFECT. Just...,2,YgavGxfAdjhkkbwlAY_9ZQ


In [25]:
min(toronto_reviews.date)

'2008-01-03'

In [24]:
toronto_reviews.to_csv('toronto/reviews_all_time.csv')

## Get all users contributing to the Toronto reviews

In [26]:
N = 16
df_matrix = [None] * N
for i in range(N):
    path = "cleaned/user%i.json" % i
    df_matrix[i] = pd.DataFrame(list(pd.read_json(path).data))

In [27]:
ALL_USERS = pd.concat(df_matrix)

In [28]:
toronto_users = ALL_USERS[ALL_USERS.user_id.isin(toronto_reviews.user_id)]

In [29]:
toronto_users.to_csv('toronto/toronto_users.csv')

## Temporal stuff (not used)

In [18]:
DF2017 = DF[('2016-31-12' < DF.date) & (DF.date < '2018-01-01')]
DF2017.to_csv('cleaned_csv/review_2017.csv')

In [19]:
len(DF2017)

1194269

In [10]:
rest = pd.read_csv('cleaned_csv/restaurants.csv')

In [12]:
rest_id = set(rest.business_id)

In [21]:
rest = pd.read_csv('cleaned_csv/restaurants.csv')

In [23]:
rest_ids = list(rest.business_id)

In [38]:
rest_reviews = DF[DF.business_id.isin(rest_ids)]

In [39]:
len(rest_reviews)

4018725

In [46]:
rest_reviews.head()

,business_id,date,review_id,stars,text,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,2011-02-25,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,2012-11-13,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,msQe1u7Z_XuqjGoqhB0J5g
2,jtQARsP6P-LbkyjbO1qNGg,2014-10-23,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,msQe1u7Z_XuqjGoqhB0J5g
3,elqbBhBfElMNSrjFqW3now,2011-02-25,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,msQe1u7Z_XuqjGoqhB0J5g
4,Ums3gaP2qM3W1XcA5r6SsQ,2014-09-05,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,msQe1u7Z_XuqjGoqhB0J5g


In [43]:
rest_reviews = rest_reviews.drop(['cool', 'funny', 'useful'], axis=1)

In [45]:
rest_reviews.to_csv('cleaned_csv/rest_reviews.csv')

In [47]:
rest_reviews_no_text = rest_reviews.drop(['text'], axis=1)

In [48]:
rest_reviews_no_text.to_csv('cleaned/rest_reviews_no_test.csv')

In [51]:
rest_reviews_no_text.date.min()

'2004-10-12'

In [52]:
rest_reviews_no_text.date.max()

'2018-07-02'

In [65]:
for year in range(2004, 2019):
    idx = rest_reviews_no_text.date.str[:4] == str(year)
    df_year = rest_reviews_no_text[idx]
    df_year.to_csv('csv_by_year/%i.csv' % year)